In [1]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
import zipfile
import random
import shutil
import numpy as np

## Folder rename

In [7]:
#Helper function
#...............................................................................
#data extraction
def data_extract(zip_f_path,extract_path,Flag=False):
    
    
    #zip_f_path contain the zipfile location
    #extract_path contain the extracting location
    Extract=Flag
    if Extract:
        
        
        local_zip =zip_f_path 
        zip_ref = zipfile.ZipFile(local_zip, 'r')
        zip_ref.extractall(extract_path)
        zip_ref.close()
    else:
        print("File already downloaded")

#...............................................................................

#Rename folder
def frenam(mapfilep, fpath):
    
    # mapfilep is the path of word file to map name
    # fpath is the path of the folder 
   
    text = open(mapfilep, 'r')
    lines = text.readlines()
    names = {}
    
    for line in lines:
        k = line.split('\t')[0]
        v = line.split('\t')[1].split(',')[0].replace('\n', '')

        names[k] = v
    _dir = fpath
    folders = os.listdir(_dir)
    try:
        for fn in folders:
            if os.path.isdir(os.path.join(_dir, fn)):
                os.rename(os.path.join(_dir, fn), os.path.join(_dir, names[fn]))
        return print("Complete")
    except:
        return print('Already done')
#...............................................................................

#Validation set processing

def val_process(source, dest, txtfpath):
    ## making dataframe to group file name based on label of folder
    import pandas as pd
    df = pd.read_csv(txtfpath, sep=" ", header=None)
    for i in range(10000):
        df.loc[i, 'val1'] = df.iloc[i, 0].split("\t")[0]
        df.loc[i, 'val2'] = df.iloc[i, 0].split("\t")[1]
    a = df.drop([0], axis=1).groupby(['val2', 'val1']).size().to_frame()

    ##Converting df to dictionary
    dic = {}
    li = []
    for i in range(len(a.index)):
        if i == len(a.index) - 1:
            li.append(a.index[i][1])
            dic[a.index[i][0]] = li
        else:
            if a.index[i][0] == a.index[i + 1][0]:
                li.append(a.index[i][1])
            else:
                li.append(a.index[i][1])
                dic[a.index[i][0]] = li
                li = []
    ##making path for destination
    if os.path.exists(dest):
        print("May be your file already processed,if not then delete the folder")


    else:
        os.mkdir(dest)
        ##copying file from source to destination
        for k, l in dic.items():

            try:
                shutil.rmtree(dest + k + '/')
                os.mkdir(dest + k + '/')
                file = l
                for li_valu in file:

                    for f in os.listdir(source):

                        if li_valu == f:
                            desfile = dest + k + '/' + f
                            thisfile = source + f
                            copyfile(thisfile, desfile)

            except:

                os.mkdir(dest + k + '/')
                file = l
                for li_valu in file:

                    for f in os.listdir(source):

                        if li_valu == f:
                            desfile = dest + k + '/' + f
                            thisfile = source + '/' + f
                            copyfile(thisfile, desfile)

        print("Completed")
#...............................................................................

# Mdodel building
def tiny_model(epoch,tbatch,vbatch,trainpath,valpath):
    
  
    #epoch is the number of iteration
    #tbatch train batch size
    #vbatch validation batch size
    #trainpath training data path
    #valpath validation data path

    #model 
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64,(3,3), input_shape=(64, 64, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512,(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(200, activation='softmax')
  ])
    model.compile(optimizer=RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['acc'])
  
  # label generation 
    TRAINING_DIR = trainpath
    train_datagen = ImageDataGenerator(
      rescale=1 / 255,
  )
    train_generator = train_datagen.flow_from_directory(
      TRAINING_DIR,
      batch_size=tbatch,
      class_mode='categorical',
      target_size=(64, 64)
        )

    VALIDATION_DIR =valpath
    validation_datagen = ImageDataGenerator(
       rescale=1 / 255
  )
    validation_generator = validation_datagen.flow_from_directory(
      VALIDATION_DIR,
      batch_size=vbatch,
      class_mode='categorical',
      target_size=(64,64)
  )
    model.summary()
    history = model.fit_generator(train_generator,
                              epochs=epoch,
                              verbose=1,steps_per_epoch=100000/tbatch,
                              validation_data=validation_generator)
    return history
#..............................................................................

## Extracting File


In [3]:
#extracting data
zip_f_path='/content/drive/MyDrive/task/archive.zip'
extract_path='/content/drive/MyDrive/task/dataset'
data_extract(zip_f_path,extract_path,Flag=False)

File already downloaded


## File Processing

In [4]:
## folder rename of training set
mp="tiny-imagenet-200/words.txt"
fpath="tiny-imagenet-200/train"
names=frenam(mp,fpath)
## validation processing
source='tiny-imagenet-200/val/images/'
dest='tiny-imagenet-200/newval/'
txtfpath='tiny-imagenet-200/val/val_annotations.txt'
val_process(source,dest,txtfpath)
##rename folder of validation
mp="tiny-imagenet-200/words.txt"
fpath=dest
frenam(mp,fpath) 



Already done
May be your file already processed,if not then delete the folder
Already done


## Training

In [ ]:
epoch=8
tbatch=128
vbatch=32
trainpath="tiny-imagenet-200/train"
valpath='tiny-imagenet-200/newval/'

history=tiny_model(epoch,tbatch,vbatch,trainpath,valpath)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 29, 29, 128)       73856     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 6, 6, 256)         0         
_____________________________

## Visualization

In [ ]:
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')
